In [2]:
import pandas as pd
from scipy import stats
import numpy as np
from itertools import product
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv(snakemake.input[0], index_col=0)

df_decomp = df[['CloudEff','DirectEff']]

df_decomp['residual'] = df.loc[:,'ERFt'] - df.loc[:,'CloudEff'] - df.loc[:,'DirectEff']

df_decomp = df_decomp.dropna()

df_decomp = df_decomp.drop('Multi-model', axis=0)

In [4]:
combinations = list(product(df_decomp['CloudEff'], df_decomp['DirectEff'],df_decomp['residual']))
            
erf = np.zeros(len(combinations))

for i,c in enumerate(combinations):
    erf[i] = sum(c)

erf=np.concatenate((erf,df.loc[~df.index.isin(df_decomp.index)]['ERFt'].values))

erf = (erf,)

boot_strap_ci = stats.bootstrap(erf, np.mean,method='basic')

boot_strap_ci.confidence_interval

In [5]:
results = {}
results['ERFt'] = {}
results['ERFt']['mean'] = erf[0].mean()
results['ERFt']['ci_low'] = boot_strap_ci.confidence_interval.low
results['ERFt']['ci_high'] = boot_strap_ci.confidence_interval.high


In [9]:
fig = plt.figure(figsize=(10,6))
pd.plotting.bootstrap_plot(pd.Series(erf[0]), fig=fig);

In [10]:
combinations_feedbacks = list(product(df['feedback_tot_dCdT'].dropna(),df['feedback_tot_dFdC'].dropna()))

In [11]:
feedbacks = np.zeros(len(combinations_feedbacks))
for i, c in enumerate(combinations_feedbacks):
    feedbacks[i] = c[0]*c[1]

In [13]:
df['feedback_tot_dCdT'].dropna()

In [9]:
boot_strap_feedback= stats.bootstrap((feedbacks,),np.mean,method='basic')

In [10]:
results['alpha']={}
results['alpha']['mean'] = feedbacks.mean()
results['alpha']['ci_low'] = boot_strap_feedback.confidence_interval.low
results['alpha']['ci_high'] = boot_strap_feedback.confidence_interval.high

In [11]:
alpha_series=pd.Series(feedbacks,name='Feedback parameter')

In [12]:
fig = plt.figure(figsize=(10,6))
pd.plotting.bootstrap_plot(alpha_series, fig=fig);

In [13]:
pd.DataFrame(results).to_csv(snakemake.output.outpath)

In [33]:
fig ,(ax,axt) = plt.subplots(ncols =2, figsize=(8,4))
ax.boxplot(erf[0],bootstrap=10000, showmeans=True, flierprops={"marker": "x"}, notch=True, whis=[5, 95])
# axt = ax.twinx()
axt.boxplot(feedbacks, bootstrap=10000, showmeans=True, flierprops={"marker": "x"}, notch=True, whis=[5, 95])
axt.xaxis.set_ticklabels(['Dust Feedback paramter'],fontsize=12)
ax.xaxis.set_ticklabels(['DRE'],fontsize=12)
axt.yaxis.set_ticks_position('right')
fig.suptitle('Dust forcing and Feedback parameter',fontsize=16)
ax.set_ylabel('$W/m^2$',fontsize=12)
# axt.set_ylabel()
axt.yaxis.set_label_text('$W/m^2 K$', fontsize=12)
axt.yaxis.set_label_position('right')
axt.set_ylim(-0.1, 0.1)
ax.set_ylim(-0.7, 0.7)
ax.text(0.05, 0.92, f'Mean: {results["ERFt"]["mean"]:.3f} W m-2 [{results["ERFt"]["ci_low"]:.3f},{results["ERFt"]["ci_high"]:.3f}]', 
        transform=ax.transAxes)
axt.text(0.05, 0.92, f'Mean: {results["alpha"]["mean"]:.3f} W m-2 [{results["alpha"]["ci_low"]:.3f},{results["alpha"]["ci_high"]:.3f}]', 
        transform=axt.transAxes)
plt.savefig(snakemake.output.outplot, dpi=144, bbox_inches='tight')